In [1]:
import csv
import numpy as np
import math
import pandas as pd

In [151]:
file = open("diabetes.csv", "r")
dataset = csv.reader(file)#đọc dữ liệu đầu vào
dataset = np.array(list(dataset))#chuyển về dạng ma trận
dataset = np.delete(dataset,0,0)#xóa header
file.close()
#Phân chia tập dữ liệu về tập Test và tập Training
dataset = dataset.astype(float)
trainingData = dataset[:538]#Tập training bao gồm 500 bản ghi dữ liệu đầu tiên
testData = dataset[538:]#Tập test bao gồm các bản ghi dữ liệu còn lại

In [152]:
print(pd.DataFrame(trainingData,columns=["Pregnancies","Glucose","BloodPressure","SkinThickness","Insulin","BMI","DiabetesPedigreeFunction","Age","Outcome"]))

     Pregnancies  Glucose  BloodPressure  SkinThickness  Insulin   BMI  \
0            6.0    148.0           72.0           35.0      0.0  33.6   
1            1.0     85.0           66.0           29.0      0.0  26.6   
2            8.0    183.0           64.0            0.0      0.0  23.3   
3            1.0     89.0           66.0           23.0     94.0  28.1   
4            0.0    137.0           40.0           35.0    168.0  43.1   
..           ...      ...            ...            ...      ...   ...   
533          6.0     91.0            0.0            0.0      0.0  29.8   
534          1.0     77.0           56.0           30.0     56.0  33.3   
535          4.0    132.0            0.0            0.0      0.0  32.9   
536          0.0    105.0           90.0            0.0      0.0  29.6   
537          0.0     57.0           60.0            0.0      0.0  21.7   

     DiabetesPedigreeFunction   Age  Outcome  
0                       0.627  50.0      1.0  
1                

In [153]:
# ko dc viet = dataset[0] do no se la 2 bien tham chieu den dataset[0]
max_item = np.array(dataset[0][:8])
min_item = np.array(dataset[0][:8])
for item in dataset:
    for i in range(8):
            if item[i] > max_item[i]:
                max_item[i] = item[i]
            if item[i] < min_item[i]:
                min_item[i] = item[i]
print(max_item)
print(min_item)
max_item = max_item.astype(float)
min_item = min_item.astype(float)

[ 17.   199.   122.    99.   846.    67.1    2.42  81.  ]
[ 0.     0.     0.     0.     0.     0.     0.078 21.   ]


In [154]:
for item in dataset:
    for i in range(8):
        item_value = float(item[i])
        if i == 1 and item_value < 100:
            item[1] = 0
        elif i == 1 and item_value > 180:
            item[1] = 1
        elif i == 5 and item_value < 22:
            item[5] = 0
        else:
            item[i] = float((item_value - min_item[i])/(max_item[i] - min_item[i]))

In [155]:
print(dataset)

[[0.35294118 0.74371859 0.59016393 ... 0.23441503 0.48333333 1.        ]
 [0.05882353 0.         0.54098361 ... 0.11656704 0.16666667 0.        ]
 [0.47058824 1.         0.52459016 ... 0.25362938 0.18333333 1.        ]
 ...
 [0.29411765 0.6080402  0.59016393 ... 0.07130658 0.15       0.        ]
 [0.05882353 0.63316583 0.49180328 ... 0.11571307 0.43333333 1.        ]
 [0.05882353 0.         0.57377049 ... 0.10119556 0.03333333 0.        ]]


In [156]:
def computeDistance(data1,data2):
  res = 0;
  for i in range(8):
    res += ( float(data1[i]) - float(data2[i])  )**2
  return math.sqrt(res)


In [157]:
def new_predict(label):
  weight1 = 0
  weight2 = 0
  for element in label:
    if int(element["label"]) == 1:
      weight1 += 1/float(element["dis"])
    else:
      weight2 += 1/float(element["dis"])
  if weight1 > weight2: 
    return 1
  else:
    return 0

In [158]:
def new_computeKNN(trainingData,point,k):
  distance = []
  for data in trainingData:
    #khai bao dictionary
    label_distance = {
        "label" : data[-1],
        "dis": computeDistance(data,point)
    }
    distance.append(label_distance)
  distance.sort(key = lambda x: x["dis"])

  return distance[:k]

In [159]:
def evaluate(x,y):
    tp = 0
    fn = 0
    fp = 0
    tn = 0
    if (x==1 and y == 1):
        tp = 1
    elif (x == 1 and y == 0):
        fn = 1
    elif (y == 1):
        fp = 1
    else:
        tn = 1
    return [tp,fn,fp,tn]

In [160]:
data_size = len(testData)
print(data_size)

230


In [165]:
k = 22
TP = 0
FN = 0
FP = 0
TN = 0

for element in testData:
    if element[1] == 0 or element[5] == 0:
        predict_label = 0
    elif element[1] == 1:
        predict_label = 1
    else:
        label = new_computeKNN(trainingData, element, k)
    #print("True = ", element[-1], "Predict = ", predict(label))
        predict_label = new_predict(label)
    tp, fn, fp, tn = evaluate(int(element[-1]), predict_label)
    TP += tp
    FN += fn
    FP += fp
    TN += tn
#print(data_size)
print("TP =", TP) # True Positive: người bị bệnh được dự đoán là bị bệnh
print("FN =", FN) # False Negative: người bị bệnh được dự đoán là không có bệnh
print("FP =", FP) # False Positive: người không bị bệnh được dự đoán là bị bệnh
print("TN =", TN) # True Negative: người không bị bệnh được dự đoán là không bị bệnh

accuracy =round(float( (TP + TN)/data_size),2)
precision = float(TP/(TP+FP)) # tính tỉ lệ số người thực sự bị bệnh trong những người được dự đoán bị bệnh
recall = float(TP/(TP+FN)) #tính tỉ lệ dự đoán chính xác trong những người bị bệnh

print(f"Accuracy: {accuracy*100:.2f}%")
print(f"Precision: {precision*100:.2f}%")
print(f"Recall: {recall*100:.2f}%")

TP = 49
FN = 30
FP = 14
TN = 137
Accuracy: 81.00%
Precision: 77.78%
Recall: 62.03%


In [149]:
data_points = []
maxa = 0
ide = 0 
for k in range(1,51):
    TP = 0
    FN = 0
    FP = 0
    TN = 0

    for element in testData:
        if element[1] == 0:
            predict_label = 0
        elif element[1] == 1:
            predict_label = 1
        else:
            
    #print("True = ", element[-1], "Predict = ", predict(label))
            predict_label = new_predict(new_computeKNN(trainingData, element, k))
    tp, fn, fp, tn = evaluate(int(element[-1]), predict_label)
    TP += tp
    FN += fn
    FP += fp
    TN += tn
    accuracy =round(float( (TP + TN)/data_size),2)
    if accuracy > maxa:
        maxa = accuracy
        ide = k
    data_points.append([k,accuracy])

In [150]:
print(maxa)

0


In [31]:
print(ide)

0
